# Cluster-based Input Weight Initialization for Echo State Networks

This notebook aims to be the supplemental material for the corresponding journal article.

We aim to pre-train the input weight matrix of ESNs using the K-Means algorithm since passing features to the non-linear reservoir of ESNs is closely related to compute the dot product between two vectors.

We use various datasets from https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

In [ ]:
import os, sys
cwd = os.getcwd()
module_path = os.path.dirname(cwd)  # target working directory

sys.path = [item for item in sys.path if item != module_path]  # remove module_path from sys.path
sys.path.append(module_path)  # add module_path to sys.path

import time
import glob
import os
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid, cross_val_score
from sklearn.utils import shuffle
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from sklearn.cluster import MiniBatchKMeans
from joblib import dump, load
from pyrcn.echo_state_network import SeqToLabelESNClassifier, SeqToSeqESNClassifier
from pyrcn.base import PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from pyrcn.model_selection import SequentialSearchCV
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
%matplotlib inline
#Options
plt.rc('image', cmap='RdBu')
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker

## Arabic spoken digit dataset 

(https://archive.ics.uci.edu/ml/datasets/Spoken+Arabic+Digit)

- Contains 6600 training utterances 
- Contains 2200 test utterances

Dataset from 8800(10 digits x 10 repetitions x 88 speakers) time series of 13 Frequency Cepstral
Coefficients (MFCCs) had taken from 44 males and 44 females Arabic native speakers
between the ages 18 and 40 to represent ten spoken Arabic digit.

Each line on the data base represents 13 MFCCs coefficients in the increasing order separated by
spaces. This corresponds to one analysis frame. The 13 Mel Frequency Cepstral Coefficients
(MFCCs) are computed with the following
conditions;
Sampling rate: 11025 Hz, 16 bits
Window applied: hamming
Filter pre-emphasized: $1-0.97^{-1}$

In [ ]:
arab = np.load(r"E:\multivariate_time_series_dataset\numpy\ARAB.npz")
X_train = np.empty(shape=(6600, ), dtype=object)
y_train = np.empty(shape=(6600, ), dtype=object)
X_test = np.empty(shape=(2200, ), dtype=object)
y_test = np.empty(shape=(2200, ), dtype=object)

for k, (X, y) in enumerate(zip(arab['X'], arab['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(arab['Xte'], arab['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)
try:
    sequential_search = load("../multidataset/sequential_search_arab.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_arab.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_arab_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_arab_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Australian Sign Language signs (High Quality) Data Set 

(https://archive.ics.uci.edu/ml/datasets/Australian+Sign+Language+signs+(High+Quality))

- Contains 1140 training utterances 
- Contains 1425 test utterances

Data was captured using a setup that consisted of:

- Two Fifth Dimension Technologies (5DT) gloves, one right and one left
- Two Ascension Flock-of-Birds magnetic position trackers, one attached to each hand
- A four-port serial card to cope with four data sources
- A PC (128MB RAM, Intel Pentium II 266MHz) was used

In terms of the quality of the data, the Flock system was far superior to the Nintendo system also available from the same donor. Firstly, this was a two-hand system. Secondly, each position tracker provided 6 degrees of freedom - i.e. roll, pitch and yaw as well as x, y and z. The gloves also provided a full five fingers of data. But the big improvements were in resolution - both accuracy and temporal. Position and orientation were defined to 14-bit accuracy, giving position information with a typical positional error less than one centimetre and angle error less than one half of a degree. Finger bend was measured with 8 bits per finger, of which probably 6 bits were usable once the glove was calibrated. The refresh rate of the complete system was close to 100 frames per second; and all signals had significantly less noise than the Nintendo data.

Samples from a single signer (a native Auslan signer) were collected over a period of nine weeks. In total, 27 samples per sign, and a total of 2565 signs were collected. The average length of each sign was approximately 57 frames.

The data was collected from a volunteer native Auslan signer 

In [ ]:
aus = np.load(r"E:\multivariate_time_series_dataset\numpy\AUS.npz")
X_train = np.empty(shape=(1140, ), dtype=object)
y_train = np.empty(shape=(1140, ), dtype=object)
X_test = np.empty(shape=(1425, ), dtype=object)
y_test = np.empty(shape=(1425, ), dtype=object)

for k, (X, y) in enumerate(zip(aus['X'], aus['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(aus['Xte'], aus['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_aus.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_aus.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 1600 - 400), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 1600,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_aus_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_aus_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 400:
        kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 400), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Character Trajectories Data Set 
(http://archive.ics.uci.edu/ml/datasets/Character+Trajectories)

Data Set Information:

The characters here were used for a PhD study on primitive extraction using HMM based models. The data consists of 2858 character samples, contained in the cell array 'mixout'. The struct variable 'consts' contains a field consts.charlabels which provides ennummerated labels for the characters. consts.key provides the key for each label. The data was captured using a WACOM tablet. 3 Dimensions were kept - x, y, and pen tip force. The data has been numerically differentiated and Gaussian smoothed, with a sigma value of 2. Data was captured at 200Hz. The data was normalised with consts.datanorm. Only characters with a single 'PEN-DOWN' segment were considered. Character segmentation was performed using a pen tip force cut-off point. The characters have also been shifted so that their velocity profiles best match the mean of the set.

Attribute Information:

Each character sample is a 3-dimensional pen tip velocity trajectory. This is contained in matrix format, with 3 rows and T columns where T is the length of the character sample.

In [ ]:
char = np.load(r"E:\multivariate_time_series_dataset\numpy\CHAR.npz")
X_train = np.empty(shape=(300, ), dtype=object)
y_train = np.empty(shape=(300, ), dtype=object)
X_test = np.empty(shape=(2558, ), dtype=object)
y_test = np.empty(shape=(2558, ), dtype=object)

for k, (X, y) in enumerate(zip(char['X'], char['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(char['Xte'], char['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_char.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_char.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 50), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_char_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_char_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 50:
        kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 50), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   ChlorineConcentration: Chlorine concentration data set 
(https://rdrr.io/github/moviedo5/fda.tsc/man/ChlorineConcentration.html)

Data Set Information:

The data set consists of 166 nodes (pipe junctions) and measurement of the Chlorine concentration level at all these nodes during 15 days (one measurement for every 5 minutes, a total of 4310 time ticks). 

This dataset was defined in a PhD thesis by Lei Li (Carnegie Mellon University). It was produced by EPANET that models the hydraulic and water quality behavior of water distribution piping systems. EPANET can track, in a given water network, the water level and pressure in each tank, the water flow in the pipes and the concentration of a chemical species (Chlorine in this case) throughout the network within a simulated duration. 

Attribute Information:



- class: Corresponding class level of “ChlorineConcentration” curves with 3 classes with 1000, 1000 and 2307 observations per class respectively.

- sample:Factor variable. In TSC database, the first 467 values (sample=train) are used for training sample and the rest of 3840 (sample=test) for testing.

In [ ]:
chlo = np.load(r"E:\multivariate_time_series_dataset\numpy\CHLO.npz")
X_train = np.empty(shape=(467, ), dtype=object)
y_train = np.empty(shape=(467, ), dtype=object)
X_test = np.empty(shape=(3840, ), dtype=object)
y_test = np.empty(shape=(3840, ), dtype=object)

for k, (X, y) in enumerate(zip(chlo['X'], chlo['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(chlo['Xte'], chlo['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_chlo.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_chlo.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_chlo_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_chlo_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  CMU Graphics Lab Motion Capture Database
(http://mocap.cs.cmu.edu/)

Data Set Information:

The mocap lab in the basement of Wean contains 12 Vicon infrared MX-40 cameras, each of which is capable of recording 120 Hz with images of 4 megapixel resolution. The cameras are placed around a rectangular area, of approximately 3m x 8m, in the center of the room. Only motions that take place in this rectangle can be captured. If motion of human hands is being captured, more detail is required and the cameras are moved closer to capture a smaller space with higher resolution.

To capture something, small grey markers are placed on it. Humans wear a black jumpsuit and have 41 markers taped on. The Vicon cameras see the markers in infra-red. The images that the various cameras pick up are triangulated to get 3D data.

Attribute Information:

 ViconIQ requires user interaction to start off the skeleton fitting. To process a capture, a segment of motion is loaded onscreen as a point cloud of markers. The user goes through and specifies the correspondence between these markers and the markers in the .vst, e.g. "this white dot is the clavicle marker". From this data ViconIQ can fit a skeleton and determine the skeleton's limb lengths. From here on out the labeling process is automatic. ViconIQ can load up each motion clip and automatically perform a "Kinematic Fit" of the skeleton to the markers. During this time the software uses its knowledge of the skeleton to correct captured marker aberrations. The user can also fix things up by editing the joint rotation/translation graphs directly. 

In [ ]:
cmu = np.load(r"E:\multivariate_time_series_dataset\numpy\CMU.npz")
X_train = np.empty(shape=(29, ), dtype=object)
y_train = np.empty(shape=(29, ), dtype=object)
X_test = np.empty(shape=(29, ), dtype=object)
y_test = np.empty(shape=(29, ), dtype=object)

for k, (X, y) in enumerate(zip(cmu['X'], cmu['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(cmu['Xte'], cmu['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_cmu.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_cmu.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_cmu_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_cmu_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  ECG dataset

Data Set Information:

TODO

Attribute Information:

TODO

In [ ]:
ecg = np.load(r"E:\multivariate_time_series_dataset\numpy\ECG.npz")
X_train = np.empty(shape=(100, ), dtype=object)
y_train = np.empty(shape=(100, ), dtype=object)
X_test = np.empty(shape=(100, ), dtype=object)
y_test = np.empty(shape=(100, ), dtype=object)

for k, (X, y) in enumerate(zip(ecg['X'], ecg['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(ecg['Xte'], ecg['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_ecg.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_ecg.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_ecg_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_ecg_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   Japanese Vowels Data Set 

Data Set Information:

The data was collected for examining our newly developed classifier for multidimensional curves (multidimensional time series). Nine male speakers uttered two Japanese vowels /ae/ successively. For each utterance, with the analysis parameters described below, we applied 12-degree linear prediction analysis to it to obtain a discrete-time series with 12 LPC cepstrum coefficients. This means that one utterance by a speaker forms a time series whose length is in the range 7-29 and each point of a time series is of 12 features (12 coefficients).

The number of the time series is 640 in total. We used one set of 270 time series for training and the other set of 370 time series for testing.

- Number of Instances (Utterances):
    - Training: 270 (30 utterances by 9 speakers. See file 'size_ae.train'.)
    - Testing: 370 (24-88 utterances by the same 9 speakers in different opportunities. See file 'size_ae.test'.)

- Length of Time Series:
     - 7 - 29 depending on utterances

- Analysis parameters:
    - Sampling rate : 10kHz
    - Frame length : 25.6 ms
    - Shift length : 6.4ms
    - Degree of LPC coefficients : 12 

Attribute Information:

12 Real Attributes

In [ ]:
jpvow = np.load(r"E:\multivariate_time_series_dataset\numpy\JPVOW.npz")
X_train = np.empty(shape=(270, ), dtype=object)
y_train = np.empty(shape=(270, ), dtype=object)
X_test = np.empty(shape=(370, ), dtype=object)
y_test = np.empty(shape=(370, ), dtype=object)

for k, (X, y) in enumerate(zip(jpvow['X'], jpvow['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(jpvow['Xte'], jpvow['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_jpvow.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_jpvow.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 50), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_jpvow_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_jpvow_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 50:
        kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 50), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   Kicks vs. Punch Dataset
(https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7813879/)

Data Set Information:

TODO

Attribute Information:

TODO

In [ ]:
kick = np.load(r"E:\multivariate_time_series_dataset\numpy\KICK.npz")
X_train = np.empty(shape=(16, ), dtype=object)
y_train = np.empty(shape=(16, ), dtype=object)
X_test = np.empty(shape=(10, ), dtype=object)
y_test = np.empty(shape=(10, ), dtype=object)

for k, (X, y) in enumerate(zip(kick['X'], kick['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(kick['Xte'], kick['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_kick.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_kick.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 100), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_kick_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_kick_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 100:
        kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 100), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##    Libras Movement Data Set
(https://archive.ics.uci.edu/ml/datasets/Libras+Movement)

Data Set Information:

The dataset (movement_libras) contains 15 classes of 24 instances each, where each class references to a hand movement type in LIBRAS.

In the video pre-processing, a time normalization is carried out selecting 45 frames from each video, in according
to an uniform distribution. In each frame, the centroid pixels of the segmented objects (the hand) are found, which
compose the discrete version of the curve F with 45 points. All curves are normalized in the unitary space.

In order to prepare these movements to be analysed by algorithms, we have carried out a mapping operation, that is, each
curve F is mapped in a representation with 90 features, with representing the coordinates of movement.

Some sub-datasets are offered in order to support comparisons of results. 

Attribute Information:

90 numeric (double) and 1 for the class (integer)

In [ ]:
lib = np.load(r"E:\multivariate_time_series_dataset\numpy\LIB.npz")
X_train = np.empty(shape=(180, ), dtype=object)
y_train = np.empty(shape=(180, ), dtype=object)
X_test = np.empty(shape=(180, ), dtype=object)
y_test = np.empty(shape=(180, ), dtype=object)

for k, (X, y) in enumerate(zip(lib['X'], lib['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(lib['Xte'], lib['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_lib.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_lib.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 1600 - 400), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 1600,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_lib_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_lib_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 400:
        kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 400), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##    NetFlow dataset


Data Set Information:

TODO

Attribute Information:

TODO


In [ ]:
net = np.load(r"E:\multivariate_time_series_dataset\numpy\NET.npz")
X_train = np.empty(shape=(803, ), dtype=object)
y_train = np.empty(shape=(803, ), dtype=object)
X_test = np.empty(shape=(534, ), dtype=object)
y_test = np.empty(shape=(534, ), dtype=object)

for k, (X, y) in enumerate(zip(net['X'], net['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(net['Xte'], net['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 4,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_net.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_net.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 100), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 4,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_net_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_net_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 100:
        kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 100), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

## PEMS-SF Data Set


Data Set Information:

15 months worth of daily data from the California Department of Transportation PEMS website, (http://pems.dot.ca.gov/), the data describes the occupancy rate, between 0 and 1, of different car lanes of San Francisco bay area freeways. The measurements cover the period from Jan. 1st 2008 to Mar. 30th 2009 and are sampled every 10 minutes. We consider each day in this database as a single time series of dimension 963 (the number of sensors which functioned consistently throughout the studied period) and length 6 x 24=144. We remove public holidays from the dataset, as well as two days with anomalies (March 8th 2009 and March 9th 2008) where all sensors were muted between 2:00 and 3:00 AM. This results in a database of 440 time series.

The task we propose on this dataset is to classify each observed day as the correct day of the week, from Monday to Sunday, e.g. label it with an integer in {1,2,3,4,5,6,7}.

I will keep separate copies of this database on my website in a Matlab format. If you use Matlab, it might be more convenient to consider these .mat files directly.

There are two files for each fold, the data file and the labels file. We have split the 440 time series between train and test folds, but you are of course free to merge them to consider a different cross validation setting.
- The PEMS_train textfile has 263 lines. Each line describes a time-series provided as a matrix. The matrix syntax is that of Matlab, e.g. [ a b ; c d] is the matrix with row vectors [a b] and [c d] in that order. Each matrix describes the different occupancies rates (963 lines, one for each station/detector) sampled every 10 minutes during the day (144 columns).
- The PEMS_trainlabel text describes, for each day of measurements described above, the day of the week on which the data was sampled, namely an integer between 1 (Mon.) and 7 (Sun.).

- PEMS_test and PEMS_testlabels are formatted in the same way, except that there are 173 test instances.

- The permutation that I used to shuffle the dataset is given in the randperm file. If you need to rearrange the data so that it follows the calendar order, you should merge train and test samples and reorder them using the inverse permutation of randperm.

Attribute Information:

Each attribute describes the measurement of the occupancy rate (between 0 and 1) of a captor location as recorded by a measuring station, at a given timestamp in time during the day. The ID of each station is given in the stations_list text file. For more information on the location (GPS, Highway, Direction) of each station please refer to the PEMS website. There are 963 (stations) x 144 (timestamps) = 138.672 attributes for each record.

In [ ]:
pems = np.load(r"E:\multivariate_time_series_dataset\numpy\PEMS.npz")
X_train = np.empty(shape=(267, ), dtype=object)
y_train = np.empty(shape=(267, ), dtype=object)
X_test = np.empty(shape=(173, ), dtype=object)
y_test = np.empty(shape=(173, ), dtype=object)

for k, (X, y) in enumerate(zip(pems['X'], pems['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(pems['Xte'], pems['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_pems.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_pems.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_pems_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_pems_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

## DistalPhalanxTW Data Set


Data Set Information:

This series of 11 classification problems were created as part of Luke Davis's PhD titled "Predictive Modelling of Bone Ageing". They are all derived from the same images, extracted from Cao et al. "Digital hand atlas and web-based bone age assessment: system design and implementation". They are designed to test the efficacy of hand and bone outline detection and whether these outlines could be helpful in bone age prediction. Algorithms to automatically extract the hand outlines and then the outlines of three bones of the middle finger (proximal, middle and distal phalanges) were applied to over 1300 images, and three human evaluators labelled the output of the image outlining as correct or incorrect. This generated three classification problems: DistalPhalanxOutlineCorrect; MiddlePhalanxOutlineCorrect; and ProximalPhalanxOutlineCorrect. The next stage of the project was to use the outlines to predict information about the subjects age. The three problems {\em DistalPhalanxOutlineAgeGroup, MiddlePhalanxOutlineAgeGroup and ProximalPhalanxOutlineAgeGroup} involve using the outline of one of the phalanges to predict whether the subject is one of three age groups: 0-6 years old, 7-12 years old and 13-19 years old. Note that these problems are aligned by subject, and hence can be treated as a multi dimensional TSC problem. Problem Phalanges contains the concatenation of all three problems. Bone age estimation is usually performed by an expert with an algorithm called Tanner-Whitehouse. This involves scoring each bone into one of seven categories based on the stage of development. The final three bone image classification problems, DistalPhalanxTW, MiddlePhalanxTW and ProximalPhalanxTW}, involve predicting the Tanner-Whitehouse score (as labelled by a human expert) from the outline.

Attribute Information:

TODO

In [ ]:
phal = np.load(r"E:\multivariate_time_series_dataset\numpy\PHAL.npz")
X_train = np.empty(shape=(400, ), dtype=object)
y_train = np.empty(shape=(400, ), dtype=object)
X_test = np.empty(shape=(139, ), dtype=object)
y_test = np.empty(shape=(139, ), dtype=object)

for k, (X, y) in enumerate(zip(phal['X'], phal['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(phal['Xte'], phal['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_phal.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_phal.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = MinMaxScaler(feature_range=(-1, 1)).fit_transform(kmeans.cluster_centers_)

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_phal_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_phal_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Robot Execution Failures Data Set 
(http://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures)

Data Set Information:

The donation includes 5 datasets, each of them defining a different learning problem:

- LP1: failures in approach to grasp position
- LP2: failures in transfer of a part
- LP3: position of part after a transfer failure
- LP4: failures in approach to ungrasp position
- LP5: failures in motion with part

In order to improve classification accuracy, a set of five feature transformation strategies (based on statistical summary features, discrete Fourier transform, etc.) was defined and evaluated. This enabled an average improvement of 20% in accuracy. The most accessible reference is Seabra Lopes and Camarinha-Matos, 1998.

Attribute Information:

All features are numeric although they are integer valued only. Each feature represents a force or a torque measured after failure detection; each failure instance is characterized in terms of 15 force/torque samples collected at regular time intervals starting immediately after failure detection; The total observation window for each failure instance was of 315 ms.

Each example is described as follows:

class
Fx1 Fy1 Fz1 Tx1 Ty1 Tz1
Fx2 Fy2 Fz2 Tx2 Ty2 Tz2
......
Fx15 Fy15 Fz15 Tx15 Ty15 Tz15

where Fx1 ... Fx15 is the evolution of force Fx in the observation window, the same for Fy, Fz and the torques; there is a total of 90 features.

In [ ]:
robot = np.load(r"E:\multivariate_time_series_dataset\numpy\ROBOT.npz")
X_train = np.empty(shape=(100, ), dtype=object)
y_train = np.empty(shape=(100, ), dtype=object)
X_test = np.empty(shape=(64, ), dtype=object)
y_test = np.empty(shape=(64, ), dtype=object)

for k, (X, y) in enumerate(zip(robot['X'], robot['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(robot['Xte'], robot['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_robot.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_robot.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_robot_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_robot_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  SwedishLeaf Data Set 
(http://www.timeseriesclassification.com/description.php?Dataset=SwedishLeaf)

Data Set Information:

Swedish leaf is a set of leaf outlines donated by Oskar S�derkvist, and used in his MSc thesis Computer vision classifcation of leaves from swedish trees, 2001. The tree classes are 1. Ulmus carpinifolia 2. Acer 3. Salix aurita 4. Quercus 5. Alnus incana 6. Betula pubescens 7. Salix alba 'Sericea' 8. Populus tremula 9. Ulmus glabra 10. Sorbus aucuparia 11. Salix sinerea 12. Populus 13.Tilia 14. Sorbus intermedia 15. Fagus silvatica 

Attribute Information:

TODO

In [ ]:
swe = np.load(r"E:\multivariate_time_series_dataset\numpy\SWE.npz")
X_train = np.empty(shape=(500, ), dtype=object)
y_train = np.empty(shape=(500, ), dtype=object)
X_test = np.empty(shape=(625, ), dtype=object)
y_test = np.empty(shape=(625, ), dtype=object)

for k, (X, y) in enumerate(zip(swe['X'], swe['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(swe['Xte'], swe['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_swe.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_swe.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = MinMaxScaler(feature_range=(-1, 1)).fit_transform(kmeans.cluster_centers_)

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_swe_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_swe_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  uWave Data Set 
(https://www.yecl.org/publications/liu09percom.pdf)

Data Set Information:

TODO

Attribute Information:

TODO

In [ ]:
uwav = np.load(r"E:\multivariate_time_series_dataset\numpy\UWAV.npz")
X_train = np.empty(shape=(200, ), dtype=object)
y_train = np.empty(shape=(200, ), dtype=object)
X_test = np.empty(shape=(428, ), dtype=object)
y_test = np.empty(shape=(428, ), dtype=object)

for k, (X, y) in enumerate(zip(uwav['X'], uwav['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(uwav['Xte'], uwav['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_uwav.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_uwav.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_uwav_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_uwav_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Wafer Data Set 
(http://www.timeseriesclassification.com/description.php?Dataset=Wafer)

Data Set Information:

This dataset was formatted by R. Olszewski as part of his thesis Generalized feature extraction for structural pattern recognition in time-series data at Carnegie Mellon University, 2001. Wafer data relates to semi-conductor microelectronics fabrication. A collection of inline process control measurements recorded from various sensors during the processing of silicon wafers for semiconductor fabrication constitute the wafer database; each data set in the wafer database contains the measurements recorded by one sensor during the processing of one wafer by one tool. The two classes are normal and abnormal. There is a large class imbalance between normal and abnormal (10.7% of the train are abnormal, 12.1% of the test). 

Attribute Information:

TODO

In [ ]:
waf = np.load(r"E:\multivariate_time_series_dataset\numpy\WAF.npz")
X_train = np.empty(shape=(298, ), dtype=object)
y_train = np.empty(shape=(298, ), dtype=object)
X_test = np.empty(shape=(896, ), dtype=object)
y_test = np.empty(shape=(896, ), dtype=object)

for k, (X, y) in enumerate(zip(waf['X'], waf['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(waf['Xte'], waf['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_waf.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_waf.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 200), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_waf_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_waf_km_sparse.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 200:
        kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 200), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  BasicMotions
(http://www.timeseriesclassification.com/description.php?Dataset=BasicMotions)

Data Set Information:

The data was generated as part of a student project where four students performed four activities whilst wearing a smart watch. The watch collects 3D accelerometer and a 3D gyroscope It consists of four classes, which are walking, resting, running and badminton. Participants were required to record motion a total of five times, and the data is sampled once every tenth of a second, for a ten second period.
Attribute Information:

TODO

In [ ]:
walk = np.load(r"E:\multivariate_time_series_dataset\numpy\WALK.npz")
X_train = np.empty(shape=(28, ), dtype=object)
y_train = np.empty(shape=(28, ), dtype=object)
X_test = np.empty(shape=(16, ), dtype=object)
y_test = np.empty(shape=(16, ), dtype=object)

for k, (X, y) in enumerate(zip(walk['X'], walk['Y'])):
    X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_train[k] = np.argwhere(y).ravel()
scaler = StandardScaler().fit(np.concatenate(X_train))
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

X_train, y_train = shuffle(X_train, y_train, random_state=0)

for k, (X, y) in enumerate(zip(walk['Xte'], walk['Yte'])):
    X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
    y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToLabelESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_walk.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_walk.joblib")

In [ ]:
base_esn = clone(sequential_search.best_estimator_).set_params(**{"hidden_layer_size": 200})
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToLabelESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                      **initially_fixed_params)

try:
    sequential_search = load("../multidataset/sequential_search_walk_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../multidataset/sequential_search_walk_km.joblib")

In [ ]:
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)

kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
base_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
base_esn.set_params(**{"hidden_layer_size": 200}, **constant_params)
search = RandomizedSearchCV(estimator=base_esn, param_distributions=step4_esn_params, **kwargs_step4).fit(X_train, y_train)
search.best_params_

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)

constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
base_esn.set_params(**search.best_params_)

param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
sequential_search = load("../multidataset/sequential_search_chlo_km.joblib")
sequential_search.all_best_params_, sequential_search.all_best_score_
sequential_search.best_estimator_.input_to_node.input_weights

In [ ]:
# idx = np.random.randint(0, 800, 50)
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.histplot(data=w_in, stat="count", legend=False)
# ax = sns.heatmap(data=sequential_search.best_estimator_.input_to_node.input_weights, cmap="RdBu", vmin=-1.0, vmax=1.0)
# ax.invert_yaxis()
# ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
# ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.xlabel("Neuron Index")
# plt.ylabel("Input Dimension")
plt.xlabel("Weight")
plt.ylabel("Count")
# plt.imshow(sequential_search.best_estimator_.input_to_node.input_weights.todense()[:, idx])
# plt.colorbar()
plt.savefig('KM_ESN_Input_Weight_Hist_CHLO.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])

fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='RdBu', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.97, 1.0)
sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
sm.set_array([])
plt.xlim((0, 2.05))
plt.ylim((0, 1.05))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)
fig.set_size_inches(4, 2.5)
tick_locator = ticker.MaxNLocator(5)
ax.yaxis.set_major_locator(tick_locator)
ax.xaxis.set_major_locator(tick_locator)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Leakage")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(10)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("Score")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Alpha")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(20)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()

In [ ]:
sequential_search.best_estimator_

In [ ]:
plt.plot(np.concatenate(X_train))
plt.xlim((0, 1000))